In [ ]:
import sys
sys.path.append('..')
from par_segmentation import *
from matplotlib import animation
%matplotlib notebook

In [ ]:
path = '/Users/blandt/Desktop/Data/PAR2 MAIN DATASET/Ctrl/220823_nwg338_tom4,15,60/03'
img = load_image(path + '/af_corrected.tif')
roi = np.loadtxt(path + '/ROI.txt')

In [ ]:
straights = []
rois_new = []

In [ ]:
iq = ImageQuantGradientDescent(img=img, roi=roi, sigma=3.5, descent_steps=300, rol_ave=5, 
                                lr=0.01, iterations=1, fit_outer=True, roi_knots=20, nfits=None, save_training=True)

In [ ]:
# First iteration
iq.run()
iq.adjust_roi()
roi2 = iq.roi[0]

for i in iq.saved_vars[0::10]:
    a = tf.concat((i['offsets'], i['offsets'][:, :1]), axis=1)
    offsets_spline = create_offsets_spline(i['offsets'], iq.roi_knots, iq.periodic, iq.n, iq.nfits, [roi,]).numpy()
    roi_new = offset_coordinates(roi, iq.freedom * tf.math.tanh(offsets_spline[0]))
    roi_new_interp = interp_roi(roi_new)
    rois_new.append(roi_new_interp)
    straights.append(straighten(img, roi_new_interp, 50))

In [ ]:
# Second iteration
iq.run()
iq.adjust_roi()
roi3 = iq.roi[0]

for i in iq.saved_vars[0::10]:
    a = tf.concat((i['offsets'], i['offsets'][:, :1]), axis=1)
    offsets_spline = create_offsets_spline(i['offsets'], iq.roi_knots, iq.periodic, iq.n, iq.nfits, [roi2,]).numpy()
    roi_new = offset_coordinates(roi2, iq.freedom * tf.math.tanh(offsets_spline[0]))
    roi_new_interp = interp_roi(roi_new)
    rois_new.append(roi_new_interp)
    straights.append(straighten(img, roi_new_interp, 50))

In [ ]:
# # Third iteration
# iq.run()

# for i in iq.saved_vars[0::10]:
#     a = tf.concat((i['offsets'], i['offsets'][:, :1]), axis=1)
#     offsets_spline = create_offsets_spline(i['offsets'], iq.roi_knots, iq.periodic, iq.n, iq.nfits, [roi3,]).numpy()
#     roi_new = offset_coordinates(roi3, iq.freedom * tf.math.tanh(offsets_spline[0]))
#     roi_new_interp = interp_roi(roi_new)
#     rois_new.append(roi_new_interp)
#     straights.append(straighten(img, roi_new_interp, 50))

In [ ]:
max_width = max(s.shape[1] for s in straights)
print(max_width)
max_intensity = max(np.max(s) for s in straights)

fig, ax = plt.subplots()
@widgets.interact(t=(0, len(straights)-1, 1))
def update(t=0): 
    ax.clear()
    pad_full = max_width - straights[t].shape[1]
    pad_left = int(np.ceil(pad_full/2))
    pad_right = int(pad_full/2)
    straight_padded = np.c_[max_intensity * np.ones([50, pad_left]), straights[t], 
                            max_intensity * np.ones([50, pad_right])]
    ax.imshow(straight_padded, cmap='gray', vmin=0, vmax=max_intensity)
    ax.axis('off')
fig.set_size_inches(10, 2)
fig.tight_layout()

### Animation

In [ ]:
# fig, ax = plt.subplots()
# fig.set_size_inches(10, 2)
# fig.tight_layout()

# def update(t=0): 
#     ax.clear()
#     pad_full = max_width - straights[t].shape[1]
#     pad_left = int(np.ceil(pad_full/2))
#     pad_right = int(pad_full/2)
#     straight_padded = np.c_[max_intensity * np.ones([50, pad_left]), straights[t], 
#                             max_intensity * np.ones([50, pad_right])]
#     ax.imshow(straight_padded, cmap='gray', vmin=0, vmax=max_intensity)
#     ax.axis('off')


# frames = np.r_[[0], np.arange(0, len(straights)-1,)]
# anim = animation.FuncAnimation(fig, update, frames=iter(frames), save_count=len(frames))
# writer = animation.writers['ffmpeg']
# writer = writer(fps=24, bitrate=2000)
# anim.save('animation.gif', writer=writer, dpi=200)

### Unstraightened figure

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img[170:-110, 130:-100], cmap='gray', vmin=0, vmax=max_intensity)
ax.axis('off')
ax.annotate('', xy=(0.5, -0.2), xycoords='axes fraction', xytext=(0.5, -0.05), 
            arrowprops=dict(arrowstyle="->", color='k', linewidth=2))
fig.subplots_adjust(bottom=0.15)
fig.savefig('raw_for_animation.png', dpi=600, transparent=True)

In [ ]:
from matplotlib.lines import Line2D

class LineDataUnits(Line2D):
    def __init__(self, *args, **kwargs):
        _lw_data = kwargs.pop("linewidth", 1) 
        super().__init__(*args, **kwargs)
        self._lw_data = _lw_data

    def _get_lw(self):
        if self.axes is not None:
            ppd = 72./self.axes.figure.dpi
            trans = self.axes.transData.transform
            return ((trans((1, self._lw_data))-trans((0, 0)))*ppd)[1]
        else:
            return 1

    def _set_lw(self, lw):
        self._lw_data = lw

    _linewidth = property(_get_lw, _set_lw)

In [ ]:
fig, ax = plt.subplots()
@widgets.interact(t=(0, len(straights)-1, 1))
def update(t=0): 
    ax.clear()
    ax.imshow(img[170:-110, 130:-100], cmap='gray', vmin=0, vmax=max_intensity)
    line = LineDataUnits(rois_new[t][:, 0] - 130, rois_new[t][:, 1] - 170, c='tab:cyan', linewidth=2, alpha=0.5,
                        linestyle='--')
    ax.add_line(line)
    ax.axis('off')
    ax.annotate('', xy=(0.5, -0.2), xycoords='axes fraction', xytext=(0.5, -0.05), 
                arrowprops=dict(arrowstyle="->", color='k', linewidth=2))
fig.subplots_adjust(bottom=0.15)

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(bottom=0.15)

def update(t=0): 
    ax.clear()
    ax.imshow(img[170:-110, 130:-100], cmap='gray', vmin=0, vmax=max_intensity)
    line = LineDataUnits(rois_new[t][:, 0] - 130, rois_new[t][:, 1] - 170, c='tab:cyan', linewidth=2, alpha=0.5,
                        linestyle='--')
    ax.add_line(line)
    ax.axis('off')
    ax.annotate('', xy=(0.5, -0.2), xycoords='axes fraction', xytext=(0.5, -0.05), 
                arrowprops=dict(arrowstyle="->", color='k', linewidth=2))

frames = np.r_[[0], np.arange(0, len(straights)-1,)]
anim = animation.FuncAnimation(fig, update, frames=iter(frames), save_count=len(frames))
writer = animation.writers['ffmpeg']
writer = writer(fps=24, bitrate=2000)
anim.save('animation2.gif', writer=writer, dpi=200)